In [1]:
import os
import re
import pandas as pd
import numpy as np
from unidecode import unidecode

In [2]:
raw_data_path = "../data/raw/olist_order_reviews_dataset.csv"
out_path = "../data/processed/order_review_classification_ptbr.csv"

In [3]:
order_reviews = pd.read_csv(raw_data_path)

In [4]:
order_reviews.head(2)

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,NaN,NaN,2018-01-18 00:00:00,2018-01-18 21:46:59
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,NaN,NaN,2018-03-10 00:00:00,2018-03-11 03:05:13


In [5]:
def concat_text(row):
    if pd.isnull(row['review_comment_title']):
        return row['review_comment_message']
    elif pd.isnull(row['review_comment_message']):
        return row['review_comment_title']
    else:
        return row['review_comment_title'] + '. ' + row['review_comment_message']

reviews_df = order_reviews[['review_id','review_score','review_comment_title','review_comment_message']]
reviews_df = reviews_df[~((reviews_df['review_comment_title'].isna()) & (reviews_df['review_comment_message'].isna()))]
reviews_df.drop_duplicates(subset=['review_id'], inplace=True)

reviews_df['comments'] = reviews_df.apply(lambda row: concat_text(row), axis=1)
reviews_df['sentiment'] = np.where(reviews_df.review_score < 4, 'neg', 'pos')

In [6]:
for rev in reviews_df[reviews_df['review_score'] == 4].comments.sample(3).tolist():
    print(rev)
    print(re.sub(r'[^A-Za-z0-9 !:.,-?$]+', '', unidecode(rev)).strip().lower(), '\n')

boa
boa 

Produto entregue sem defeito e dentro do prazo, mas o produto não é bom; a base é de plástico e muito ruim, as argolas são ruins também.
produto entregue sem defeito e dentro do prazo, mas o produto nao e bom; a base e de plastico e muito ruim, as argolas sao ruins tambem. 

super recomendo. Recebi corretamente e até antes do esperado 
super recomendo. recebi corretamente e ate antes do esperado 



In [7]:
def fix_sentence(sentence):
    """
    keep only alphanumeric/single spaces/some puctuations and ignore non-ascii chars
    """
    return re.sub(r'[^A-Za-z0-9 !:.,-?$]+', '', unidecode(sentence)).strip().lower()

reviews_df = reviews_df[['comments','sentiment']]
reviews_df['comments'] = reviews_df['comments'].apply(fix_sentence)
reviews_df.dropna(inplace=True)
reviews_df = reviews_df[reviews_df.comments.str.replace(' ', '', regex=False).str.len() >= 2]
reviews_df.reset_index(drop=True, inplace=True)
reviews_df.head(3)

,comments,sentiment
0,recebi bem antes do prazo estipulado.,pos
1,parabens lojas lannister adorei comprar pela i...,pos
2,recomendo. aparelho eficiente. no site a marca...,pos


In [8]:
reviews_df.to_csv(out_path, index=False, sep=";")

In [1]:
import random

reviews_df['mdl_prediction'] = reviews_df.sentiment.apply(lambda sent: random.uniform(0.45, 1) if sent == 'pos' else random.uniform(0.0, 0.55))

In [3]:
random.uniform(0.5, 1)

0.6367024653465414